<a href="https://colab.research.google.com/github/Otter333/nohehe/blob/main/EX05_04_Reply_Template_Messages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 安装必要的库
!pip install -q pyngrok
!pip install -q flask
!pip install -q line-bot-sdk

# 导入模块
from flask import Flask, request, abort
from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.webhooks import MessageEvent, TextMessageContent
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage,
    TemplateMessage,
    ConfirmTemplate,
    MessageAction,
    CarouselColumn,
    URIAction,
    PostbackAction
)
from google.colab import userdata
from pyngrok import ngrok

# 初始化 Flask 应用
app = Flask(__name__)

# 配置 LINE Bot
configuration = Configuration(access_token=userdata.get('LINE_CHANNEL_ACCESS_TOKEN'))
handler = WebhookHandler(userdata.get('LINE_CHANNEL_SECRET'))

# 定义 webhook 回调
@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-LINE-SIGNATURE']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

# 处理消息事件
@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        action = event.message.text
        if action == 'confirm':
            reply = TemplateMessage(
                alt_text="這是確認視窗",
                template=ConfirmTemplate(
                    text="你喜歡葬送的芙莉蓮嗎?",
                    actions=[
                        MessageAction(label="是", text="我超喜歡芙莉蓮!"),
                        MessageAction(label="否", text="其實我非常喜歡，只是我要傲嬌的說不")
                    ]
                )
            )
        elif action == 'carousel':
            carousel_template = CarouselTemplate(
                columns=[
                    CarouselColumn(
                        thumbnail_image_url="https://upload.wikimedia.org/wikipedia/zh/7/7d/Summer_Wars_poster.jpg",
                        title="夏日大作戰",
                        text="細田守導演的日本科幻暨曼電影",
                        actions=[
                            URIAction(label="維基百科", uri="https://zh.wikipedia.org/zh-tw/夏日大戰#影片介紹")，
                            MessageAction(label="投票", text="我投夏日大作戰一票！")
                        ]
                    ),
                    CarouselColumn(
                        thumbnail_image_url="https://upload.wikimedia.org/wikipedia/zh/thumb/4/4f/Castle_of_Cagliostro_poster.png/250px-Castle_of_Cagliostro_poster.png",
                        title="魯邦三世:卡里奧斯特羅城",
                        text="宮崎駿導演的日本動畫動作冒險喜劇電影",
                        actions=[
                            URIAction(label="維基百科", uri="https://zh.wikipedia.org/wiki/魯邦三世：卡里奧斯特羅城")，
                            MessageAction(label="投票", text="我投魯邦三世:卡里奧斯特羅城一票！")
                        ]
                    )
                ]
            )
            reply = TemplateMessage(
                alt_text="輪播範例",
                template=carousel_template
            )
        else:
            reply = TextMessage(text="Thanks!")

        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[reply]
            )
        )

# 运行应用（使用 ngrok 进行端口映射）
if __name__ == "__main__":
    ngrok.set_auth_token(userdata.get('NGROK_AUTHTOKEN'))
    public_url = ngrok.connect(5000)
    print(f"ngrok URL: {public_url}")
    app.run()

ngrok URL: NgrokTunnel: "https://822a-34-75-168-67.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
